In [1]:
from py2neo import Graph, Node, Relationship
import neo4j
import csv
from datetime import datetime
from neo4j import GraphDatabase

In [2]:
driver = GraphDatabase.driver(
    "bolt://localhost:7687", #uri of neo4j
    auth=("neo4j", "Sumit@1234"),  #username and password 
)

In [3]:
session = driver.session()

In [4]:
data = open(r"C:\SUMIT\Enterprise search\dataframe_ahan_csv.csv")
data = csv.reader(data)
print(data)

In [5]:
import pandas as pd
df = pd.read_csv(r"C:\SUMIT\Enterprise search\dataframe_ahan_csv.csv")

# View the DataFrame.
print(df)

                 SentOn                                          Sender  \
0   2023-08-06 18:24:23                                  HR India (IND)   
1   2023-08-07 11:09:44                                  HR India (IND)   
2   2023-08-07 13:38:41                              Sachin Tayal (IND)   
3   2023-08-07 18:48:33                                  HR India (IND)   
4   2023-08-07 21:46:12                                  HR India (IND)   
5   2023-08-08 01:40:20          Varun Kukreja, Omar Moustafa, +1 other   
6   2023-08-08 11:06:42                                  HR India (IND)   
7   2023-08-08 11:06:53                                  HR India (IND)   
8   2023-08-08 15:05:27                     isharesupport@protiviti.com   
9   2023-08-08 15:05:31                              Protiviti Learning   
10  2023-08-08 16:06:48                            Servicedesk-HR India   
11  2023-08-08 16:40:10                                   Raya Banerjee   
12  2023-08-08 18:22:23  

In [6]:
import py2neo
graph = Graph(
    "bolt://localhost:7687",
    auth=("neo4j", "Sumit@1234"), #neo4j desktop credentials
)

In [7]:
column_dtype = df["SentOn"].dtype
print(column_dtype)

object


In [8]:
df['SentOn'] = pd.to_datetime(df['SentOn'], format='%Y-%m-%d %H:%M:%S')
column_dtype = df["SentOn"].dtype
print(column_dtype)

datetime64[ns]


In [9]:
from py2neo import Graph, Node, Relationship

# Assuming 'df' is your DataFrame and 'graph' is your Neo4j graph database connection

# Create a graph for the combined data
combined_graph = Graph("neo4j://localhost:7687", auth=("neo4j", "Sumit@1234"))

# Create nodes for SentOn, Sender, Subject, and Body
sent_on_nodes = []
sender_nodes = []
subject_nodes = []
body_nodes = []

# Create a node for each email
email_nodes = []

# Create a variable to store the previous email node
prev_email_node = None

# Create a higher-level node to connect to all the emails
all_emails_node = Node("Ahan_Mailbox")
combined_graph.create(all_emails_node)

for index, row in df.iterrows():
    sent_on_node = Node("SentOn", sent_on=row['SentOn'])
    sender_node = Node("Sender", sender=row['Sender'])
    subject_node = Node("Subject", subject=row['Subject'])
    body_node = Node("Body", body=row['Body'])

    sent_on_nodes.append(sent_on_node)
    sender_nodes.append(sender_node)
    subject_nodes.append(subject_node)
    body_nodes.append(body_node)

    combined_graph.create(sent_on_node)
    combined_graph.create(sender_node)
    combined_graph.create(subject_node)
    combined_graph.create(body_node)

    # Create a node representing the email
    email_node = Node("Email")
    email_nodes.append(email_node)
    combined_graph.create(email_node)

    # Connect the nodes to the email node
    combined_graph.create(Relationship(email_node, "SENT_ON", sent_on_node))
    combined_graph.create(Relationship(email_node, "SENT_BY", sender_node))
    combined_graph.create(Relationship(email_node, "CONTAINS_SUBJECT", subject_node))
    combined_graph.create(Relationship(email_node, "CONTAINS_BODY", body_node))

    # Connect the current email node to the previous one
    if prev_email_node:
        combined_graph.create(Relationship(prev_email_node, "FOLLOWS", email_node))

    # Connect the email node to the higher-level node
    combined_graph.create(Relationship(all_emails_node, "CONTAINS_EMAIL", email_node))

    # Update the previous email node
    prev_email_node = email_node

# Create relationships between Sender and other nodes
for i in range(len(sender_nodes)):
    subject_relationship = Relationship(sender_nodes[i], "CONTAINS_SUBJECT", subject_nodes[i])
    body_relationship = Relationship(sender_nodes[i], "CONTAINS_BODY", body_nodes[i])
    sent_on_relationship = Relationship(sender_nodes[i], "SENT_ON", sent_on_nodes[i])

    combined_graph.create(subject_relationship)
    combined_graph.create(body_relationship)
    combined_graph.create(sent_on_relationship)

# Create relationships between Subject and Body nodes
for i in range(len(subject_nodes)):
    body_relationship = Relationship(subject_nodes[i], "CONTAINS_BODY", body_nodes[i])
    sent_on_relationship = Relationship(subject_nodes[i], "SENT_ON", sent_on_nodes[i])

    combined_graph.create(body_relationship)
    combined_graph.create(sent_on_relationship)

# Create relationships between Body and SentOn nodes
for i in range(len(body_nodes)):
    sent_on_relationship = Relationship(body_nodes[i], "SENT_ON", sent_on_nodes[i])
    combined_graph.create(sent_on_relationship)

# Now, 'all_emails_node' is connected to all the individual email nodes
